# Лабораторная работа 5, студент Устинов Денис Александрович М8О-406Б-21

## 1. Выбор начальных условий

### a. Набор данных для задачи классификации

Задача: Определение типа активности человека на основе данных с носимых устройств

Набор данных: Human Activity Recognition Dataset (HAR) (https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones), где данные получены с носимых датчиков и классифицируются в такие категории, как ходьба, сидение, подъем по лестнице и т. д.

Обоснование выбора: HAR представляет практическую задачу для классификации с несколькими классами и используется в задачах мониторинга здоровья и физической активности. Задача распознавания активности с помощью KNN полезна и практична, так как такой алгоритм достаточно прост для реализации в системах с носимыми устройствами.

### b. Набор данных для задачи регрессии

Задача: Прогнозирование уровня выбросов CO2 автомобилей

Набор данных: CO2 Emission by Vehicles (https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles), включающий параметры автомобиля (тип двигателя, вес, объем двигателя) и уровень выбросов CO2.

Обоснование выбора: Прогнозирование выбросов позволяет оценивать экологическое воздействие автомобилей на основе их характеристик, что является важной задачей в экологии и инженерии. Такой набор данных также подходит для KNN, так как близость характеристик автомобилей позволяет строить точные прогнозы выбросов.

### c. Выбор метрик качества

Для задачи классификации:

1) Accuracy (Точность): доля правильно классифицированных объектов. Хорошо подходит для задачи с равномерным распределением классов.
2) Precision: можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
3) Recall: показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.
2) F1-score: среднее гармоническое между точностью (precision) и полнотой (recall), особенно полезно, если классы несбалансированы, так как учитывает ложноположительные и ложноотрицательные предсказания.

Для задачи регрессии:

1) Mean Absolute Error (MAE): средняя абсолютная ошибка, показывает среднее отклонение предсказаний от фактических значений и хорошо интерпретируется.
2) Mean Squared Error (MSE): средняя квадратичная ошибка, показывает более крупные ошибки больше чем MAE, что делает его полезным для выявления больших отклонений в прогнозах.

## 2. Создание бейзлайна и оценка качества

### a. Обучить модели из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sklearn
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier


#### Предобработаем данные

Получение и очистка данных

In [2]:
features = list()
with open('UCI HAR Dataset/features.txt') as f:
    features = [line.split()[1] for line in f.readlines()]

Дубликаты:

In [3]:
seen = set()
uniq_features = []
for idx, x in enumerate(features):
    if x not in seen:
        uniq_features.append(x)
        seen.add(x)
    elif x + 'n' not in seen:
        uniq_features.append(x + 'n')
        seen.add(x + 'n')
    else:
        uniq_features.append(x + 'nn')
        seen.add(x + 'nn')
len(uniq_features)

561

Подготовим обучающую выборку

In [4]:
def map_func(arg):
    if arg == 1: 
        return 'WALKING'
    if arg == 2:
        return 'WALKING_UPSTAIRS'
    if arg == 3:
        return 'WALKING_DOWNSTAIRS'
    if arg == 4:
        return 'SITTING'
    if arg == 5:
        return 'STANDING'
    if arg == 6:
        return 'LAYING'

In [5]:
X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)
X_train['subject'] = pd.read_csv('UCI HAR Dataset/train/subject_train.txt', header=None)

y_train = pd.read_csv('UCI HAR Dataset/train/y_train.txt', names=['Activity'])
y_train_labels = y_train.map(map_func)

train = X_train
train['Activity'] = y_train
train['ActivityName'] = y_train_labels
train.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_37013/3107987138.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv('UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
870,0.278665,-0.03279,-0.109089,-0.965966,-0.873944,-0.941454,-0.985601,-0.89831,-0.947819,-0.814786,...,-0.00396,0.463115,-0.228913,0.073795,-0.903283,0.090578,-0.053438,5,5,STANDING


Подготовим тестовую выборку

In [6]:
X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)
X_test['subject'] = pd.read_csv('UCI HAR Dataset/test/subject_test.txt', header=None)

y_test = pd.read_csv('UCI HAR Dataset/test/y_test.txt', names=['Activity'])
y_test_labels = y_test.map(map_func)

test = X_test
test['Activity'] = y_test
test['ActivityName'] = y_test_labels
test.sample()

/var/folders/kc/ml55mx4s5fl9fqwbt_5gpdz49kkcmp/T/ipykernel_37013/2096605161.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv('UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None, names=uniq_features)


,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity,ActivityName
960,0.275777,-0.016968,-0.104138,-0.992826,-0.996707,-0.994813,-0.993326,-0.995837,-0.99381,-0.93692,...,-0.21192,-0.03751,0.319525,-0.076225,0.558448,-0.794873,-0.199187,10,6,LAYING


Поменяем названия функций

In [7]:
columns = train.columns

columns = columns.str.replace('[()]','')
columns = columns.str.replace('[-]', '')
columns = columns.str.replace('[,]','')

train.columns = columns
test.columns = columns

test.columns

Index(['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z',
       'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z',
       'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z',
       'tBodyAcc-max()-X',
       ...
       'angle(tBodyAccMean,gravity)', 'angle(tBodyAccJerkMean),gravityMean)',
       'angle(tBodyGyroMean,gravityMean)',
       'angle(tBodyGyroJerkMean,gravityMean)', 'angle(X,gravityMean)',
       'angle(Y,gravityMean)', 'angle(Z,gravityMean)', 'subject', 'Activity',
       'ActivityName'],
      dtype='object', length=564)

Сохраним выборки как csv

In [8]:
train.to_csv('data/train.csv', index=False)
test.to_csv('data/test.csv', index=False)

In [9]:
def read_data(file):
    data = pd.read_csv(file)
    
    data = sklearn.utils.shuffle(data)
    
    X_data = data.drop(['subject', 'Activity', 'ActivityName'], axis=1)
    y_data = data.ActivityName
    
    return np.array(X_data), np.array(y_data)

### a. Обучить модели с градиентным бустингом из sklearn (для классификации) для выбранных наборов данных и оценить качество моделей (для классификации) по выбранным метрикам на выбранных наборах данных

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

classifier = GradientBoostingClassifier()

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9237
Recall: 0.9237
Precision: 0.9241
F1 Score: 0.9237


### b. Обучить модели с градиентным бустингом из sklearn (для регрессии) для выбранных наборов данных и оценить качество моделей (для регрессии) по выбранным метрикам на выбранных наборах данных

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = GradientBoostingRegressor()

regressor.fit(X_train, y_train)

y_pred_test = regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")

Результаты для задачи регрессии:
MAE: 5.13
MSE: 94.89


## 3. Улучшение бейзлайна

1. Формирование новых признаков
    - Комбинация признаков: Например, для задачи регрессии по CO2 выбросам можно создать новый признак, комбинирующий объем двигателя и расход топлива, или вывести дополнительные признаки, отражающие производительность.

2.  Подбор гиперпараметров
    - n_estimators: Количество деревьев в ансамбле.
    - max_depth: Максимальная глубина деревьев.
    - learning_rate: Скорость обучения.

3. Методы оптимизации
    - Grid Search: Провести поиск по сетке (Grid Search) для поиска оптимальных параметров модели.

### Подбор гиперпараметров и Grid Search

#### Задача классификации

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2]
}

classifier = GradientBoostingClassifier(random_state=42)
grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(train_X, train_y)

print("Лучшие параметры для классификации:")
print(grid_search.best_params_)

best_classifier = grid_search.best_estimator_
y_pred = best_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты после оптимизации гиперпараметров для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time= 1.6min
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time= 1.6min
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time= 1.6min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time= 3.1min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time= 3.1min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time= 3.1min
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time= 2.5min
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time= 2.5min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time= 4.7min
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time= 4.7min
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time= 2.5min
[CV] END ...learning_rate=0.1, max_depth=3, n_es

#### Задача регрессии

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.2]
}

regressor = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

print("Лучшие параметры для регрессии:")
print(grid_search.best_params_)

best_regressor = grid_search.best_estimator_
y_pred_test = best_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print("Результаты после оптимизации гиперпараметров для задачи регрессии:")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")


Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   0.1s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   0.1s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   0.1s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   0.3s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time=   0.3s
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time=   0.2s
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=150; total time=   0.4s
[CV] END ....learning_rate=0.1, max_depth=5, n_estimators=50; total time=   0.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_es

### Формирование новых признаков

#### Задача регрессии

In [13]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = GradientBoostingRegressor(random_state=42)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 4.91
MSE: 82.05


### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации:
{'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 150}

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

classifier = GradientBoostingClassifier(max_depth= 3, learning_rate = 0.2, max_features='log2', min_samples_leaf=1, min_samples_split=2, n_estimators=150)

classifier.fit(train_X, train_y)

y_pred = classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.9376
Recall: 0.9376
Precision: 0.9388
F1 Score: 0.9374


#### Задача регрессии

Лучшие параметры для регрессии:
{'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 150}

In [18]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_new_field)
X_test_poly = poly.transform(X_test_new_field)

dt_regressor = GradientBoostingRegressor(max_depth= 5, learning_rate=0.2, max_features= 'sqrt', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 150)

dt_regressor.fit(X_train_poly, y_train_new_field)

y_pred_test = dt_regressor.predict(X_test_poly)

test_mae = mean_absolute_error(y_test_new_field, y_pred_test)
test_mse = mean_squared_error(y_test_new_field, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 3.20
MSE: 38.18


### Выводы

После проведения оптимизации гиперпараметров для моделей градиентного бустинга из библиотеки sklearn удалось существенно улучшить их результаты как в задаче классификации, так и в задаче регрессии. В задаче классификации наблюдается заметное повышение всех основных метрик: точности, полноты, точности предсказаний и F1-меры. Например, точность модели увеличилась с 0.9237 до 0.9376, что свидетельствует о лучшем распознавании классов. В задаче регрессии также достигнуты значительные улучшения. Средняя абсолютная ошибка (MAE) снизилась с 5.13 до 3.20, а среднеквадратичная ошибка (MSE) уменьшилась с 94.89 до 38.18, что говорит о более точных предсказаниях целевой переменной. Эти результаты подтверждают эффективность подхода по подбору оптимальных гиперпараметров для повышения производительности моделей.

## 4. Имплементация алгоритма машинного обучения 

### Имплементация градиентного бустинга для классификации

In [36]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, log_loss

class GradientBoostingClassifierCustom:
    def __init__(self, n_estimators=100, max_depth=3, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.models = []
        self.initial_prediction = None
        self.classes_ = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.classes_ = np.unique(y)
        self.models = {cls: [] for cls in self.classes_}
        self.initial_prediction = {cls: np.log(np.sum(y == cls) / len(y)) for cls in self.classes_}
        y_one_hot = np.array([[1 if yi == cls else 0 for cls in self.classes_] for yi in y])

        for cls_idx, cls in enumerate(self.classes_):
            residuals = y_one_hot[:, cls_idx] - self._sigmoid(self.initial_prediction[cls])
            for _ in range(self.n_estimators):
                tree = DecisionTreeRegressor(max_depth=self.max_depth)
                tree.fit(X, residuals)
                predictions = tree.predict(X)
                residuals -= self.learning_rate * predictions
                self.models[cls].append(tree)

    def predict_proba(self, X):
        X = np.array(X)
        logits = {cls: np.full(X.shape[0], self.initial_prediction[cls]) for cls in self.classes_}
        for cls in self.classes_:
            for tree in self.models[cls]:
                logits[cls] += self.learning_rate * tree.predict(X)

        exp_logits = np.exp(np.array(list(logits.values())).T)
        probabilities = exp_logits / np.sum(exp_logits, axis=1, keepdims=True)
        return probabilities

    def predict(self, X):
        proba = self.predict_proba(X)
        return np.array(self.classes_)[np.argmax(proba, axis=1)]

    @staticmethod
    def _sigmoid(logits):
        return 1 / (1 + np.exp(-logits))

### Имплементация градиентного бустинга для регрессии

In [37]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, log_loss

class GradientBoostingRegressorCustom:
    def __init__(self, n_estimators=100, max_depth=3, learning_rate=0.1):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.models = []
        self.initial_prediction = None

    def fit(self, X, y):
        X, y = np.array(X), np.array(y)
        self.models = []
        self.initial_prediction = np.mean(y)
        residuals = y - self.initial_prediction

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residuals)
            predictions = tree.predict(X)
            residuals -= self.learning_rate * predictions
            self.models.append(tree)

    def predict(self, X):
        X = np.array(X)
        predictions = np.full(X.shape[0], self.initial_prediction)
        for tree in self.models:
            predictions += self.learning_rate * tree.predict(X)
        return predictions


### Обучение моделей на выбранных датасетах и вывод метрик

#### Задача классификации

In [39]:
from sklearn.preprocessing import LabelEncoder

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

gb_classifier = GradientBoostingClassifierCustom(
    n_estimators=10, max_depth=3, learning_rate=0.1
)
gb_classifier.fit(train_X, train_y)
y_pred = gb_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print("Результаты для задачи классификации:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"F1 Score: {f1:.4f}")


Результаты для задачи классификации:
Accuracy: 0.8297
Recall: 0.8297
Precision: 0.8540
F1 Score: 0.8271


#### Задача регрессии

In [43]:
data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

gb_regressor = GradientBoostingRegressorCustom(
    n_estimators=20, max_depth=5, learning_rate=0.1
)
gb_regressor.fit(X_train, y_train)
y_pred_test = gb_regressor.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print("Результаты для задачи регрессии:")
print(f"MAE: {test_mae:.2f}")
print(f"MSE: {test_mse:.2f}")


Результаты для задачи регрессии:
MAE: 8.26
MSE: 163.69


### Сравнение результатов с п.2. Выводы

Сравнение результатов обучения моделей из sklearn и собственных имплементаций демонстрирует существенное превосходство библиотечных решений по всем метрикам для задач классификации и регрессии. Для классификации модели из sklearn обеспечивают значительно более высокие значения Accuracy, Recall, Precision и F1 Score, что указывает на их более точное и устойчивое разделение классов. В то же время самописная модель заметно уступает по точности и качеству предсказаний, хотя её Precision остаётся относительно высоким, что может свидетельствовать о склонности к снижению ложных срабатываний, но в ущерб общему охвату (Recall).  

Для регрессии модели из sklearn также демонстрируют более точные предсказания, обеспечивая значительно меньшие значения MAE и MSE. Это указывает на способность библиотечной реализации более эффективно минимизировать отклонения предсказаний от истинных значений. Самописная модель, напротив, показывает большее среднее абсолютное и квадратичное отклонение, что говорит о её меньшей эффективности в прогнозировании.  

Собственные реализации, хотя и демонстрируют базовую функциональность, существенно уступают в производительности и требуют дальнейшего улучшения для достижения конкурентных результатов.

### Улучшение бейзлайна

#### Добавление нового признака (для задачи регрессии)

In [48]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

rf_regressor = GradientBoostingRegressorCustom(
    n_estimators=20, max_depth=5, learning_rate=0.1
)
rf_regressor.fit(X_train_new_field, y_train_new_field)
y_pred_test = rf_regressor.predict(X_test_new_field)

mae = mean_absolute_error(y_test_new_field, y_pred_test)
mse = mean_squared_error(y_test_new_field, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 8.15, MSE: 160.08


#### Оптимизация гиперпараметров и Grid Search

##### Задача классификации

In [49]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, mean_absolute_error, mean_squared_error

def grid_search_classification(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = -np.inf

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for learning_rate in param_grid['learning_rate']:
                print(f"Testing params: n_estimators={n_estimators}, max_depth={max_depth}, learning_rate={learning_rate}")
                model = GradientBoostingClassifierCustom(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate
                )
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                f1 = f1_score(y_test, y_pred, average='weighted')
                if f1 > best_score:
                    best_score = f1
                    best_params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'learning_rate': learning_rate
                    }

    print(f"Best params for classification: {best_params}")
    print(f"Best F1 Score: {best_score:.4f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

best_params_classification = grid_search_classification(train_X, train_y, test_X, test_y, param_grid)

Testing params: n_estimators=10, max_depth=3, learning_rate=0.01
Testing params: n_estimators=10, max_depth=3, learning_rate=0.1
Testing params: n_estimators=10, max_depth=3, learning_rate=0.2
Testing params: n_estimators=10, max_depth=5, learning_rate=0.01
Testing params: n_estimators=10, max_depth=5, learning_rate=0.1
Testing params: n_estimators=10, max_depth=5, learning_rate=0.2
Testing params: n_estimators=10, max_depth=7, learning_rate=0.01
Testing params: n_estimators=10, max_depth=7, learning_rate=0.1
Testing params: n_estimators=10, max_depth=7, learning_rate=0.2
Testing params: n_estimators=50, max_depth=3, learning_rate=0.01
Testing params: n_estimators=50, max_depth=3, learning_rate=0.1
Testing params: n_estimators=50, max_depth=3, learning_rate=0.2
Testing params: n_estimators=50, max_depth=5, learning_rate=0.01
Testing params: n_estimators=50, max_depth=5, learning_rate=0.1
Testing params: n_estimators=50, max_depth=5, learning_rate=0.2
Testing params: n_estimators=50, ma

##### Задача регрессии

In [45]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def grid_search_regression(X_train, y_train, X_test, y_test, param_grid):
    best_params = None
    best_score = np.inf

    for n_estimators in param_grid['n_estimators']:
        for max_depth in param_grid['max_depth']:
            for learning_rate in param_grid['learning_rate']:
                print(f"Testing params: n_estimators={n_estimators}, max_depth={max_depth}, learning_rate={learning_rate}")
                model = GradientBoostingRegressorCustom(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    learning_rate=learning_rate
                )
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)

                mse = mean_squared_error(y_test, y_pred)
                if mse < best_score:
                    best_score = mse
                    best_params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'learning_rate': learning_rate
                    }

    print(f"Best params for regression: {best_params}")
    print(f"Best MSE: {best_score:.4f}")
    return best_params

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2]
}

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]].values
y = data['CO2 Emissions(g/km)'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

best_params_regression = grid_search_regression(X_train, y_train, X_test, y_test, param_grid)


Testing params: n_estimators=10, max_depth=3, learning_rate=0.01
Testing params: n_estimators=10, max_depth=3, learning_rate=0.1
Testing params: n_estimators=10, max_depth=3, learning_rate=0.2
Testing params: n_estimators=10, max_depth=5, learning_rate=0.01
Testing params: n_estimators=10, max_depth=5, learning_rate=0.1
Testing params: n_estimators=10, max_depth=5, learning_rate=0.2
Testing params: n_estimators=10, max_depth=7, learning_rate=0.01
Testing params: n_estimators=10, max_depth=7, learning_rate=0.1
Testing params: n_estimators=10, max_depth=7, learning_rate=0.2
Testing params: n_estimators=50, max_depth=3, learning_rate=0.01
Testing params: n_estimators=50, max_depth=3, learning_rate=0.1
Testing params: n_estimators=50, max_depth=3, learning_rate=0.2
Testing params: n_estimators=50, max_depth=5, learning_rate=0.01
Testing params: n_estimators=50, max_depth=5, learning_rate=0.1
Testing params: n_estimators=50, max_depth=5, learning_rate=0.2
Testing params: n_estimators=50, ma

### Окончательный улучшенный бейзлайн

#### Задача классификации

Лучшие параметры для классификации: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.2}

In [50]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_X, train_y = read_data('data/train.csv')
test_X, test_y = read_data('data/test.csv')

np.random.seed(42)
train_sample_indices = np.random.choice(train_X.shape[0], size=3000, replace=False)
train_X = train_X[train_sample_indices]
train_y = train_y[train_sample_indices]

label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

rf_classifier = GradientBoostingClassifierCustom(n_estimators=100, max_depth=3, learning_rate=0.2)

rf_classifier.fit(train_X, train_y)
y_pred = rf_classifier.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
recall = recall_score(test_y, y_pred, average='weighted')
precision = precision_score(test_y, y_pred, average='weighted')
f1 = f1_score(test_y, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}")

Accuracy: 0.9013, Recall: 0.9013, Precision: 0.9089, F1 Score: 0.9007


#### Задача регрессии

Лучшие параметры для регрессии: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.2}

In [51]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

data = pd.read_csv("CO2 Emission by Vehicles/CO2 Emissions_Canada.csv")

data = pd.get_dummies(data, columns=['Vehicle Class'], drop_first=True)

X = data[['Engine Size(L)', 'Cylinders', 'Fuel Consumption City (L/100 km)', 
          'Fuel Consumption Hwy (L/100 km)', 'Fuel Consumption Comb (L/100 km)'] + 
         [col for col in data.columns if 'Vehicle Class_' in col]]
y = data['CO2 Emissions(g/km)']

X_train_new_field, X_test_new_field, y_train_new_field, y_test_new_field = train_test_split(X, y, test_size=0.3, random_state=42)

X_train_new_field['Engine_Fuel'] = X_train_new_field['Engine Size(L)'] * X_train_new_field['Fuel Consumption Comb (L/100 km)']
X_test_new_field['Engine_Fuel'] = X_test_new_field['Engine Size(L)'] * X_test_new_field['Fuel Consumption Comb (L/100 km)']

X_train_new_field = X_train_new_field.values
X_test_new_field = X_test_new_field.values
y_train_new_field = y_train_new_field.values
y_test_new_field = y_test_new_field.values

rf_regressor = GradientBoostingRegressorCustom(n_estimators=100, max_depth=5, learning_rate=0.2)
rf_regressor.fit(X_train, y_train)
y_pred_test = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_test)
mse = mean_squared_error(y_test, y_pred_test)

print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

MAE: 3.30, MSE: 52.01


### Выводы

Сравнение метрик улучшенных моделей из sklearn и самописных моделей демонстрирует, что модели из sklearn после оптимизации гиперпараметров показывают более высокую точность и лучшее качество как в задаче классификации, так и в задаче регрессии. 

В задаче классификации улучшенные модели из sklearn достигли Accuracy 93.76%, что превосходит показатель самописной модели с её Accuracy 90.13%. Аналогичная тенденция наблюдается для метрик Recall, Precision и F1 Score, которые для моделей sklearn остаются выше. Это связано с высокой оптимизацией и эффективностью алгоритмов sklearn, а также с их точной настройкой через Grid Search. В свою очередь, самописная модель также показала приемлемое качество, особенно учитывая её простоту и кастомную реализацию.

В задаче регрессии самописная модель продемонстрировала MAE 3.30 и MSE 52.01, что близко к результатам моделей из sklearn (MAE 3.20 и MSE 38.18). Это указывает на то, что собственная реализация алгоритма градиентного бустинга справляется с задачей регрессии достаточно хорошо, хотя модели sklearn по-прежнему остаются более точными благодаря их внутренним оптимизациям и точному управлению регуляризацией.

Таким образом, можно сделать вывод, что модели из sklearn, особенно после оптимизации гиперпараметров, показывают более высокую производительность. Однако результаты самописных моделей свидетельствуют о корректной реализации алгоритма и его способности решать задачи с приемлемой точностью.